In [ ]:
from pathlib import Path
import ipyparallel as ip

NODES = 16  # adjust
client = ip.Client(connection_info=str(Path("~/nfs/security/ipcontroller-client.json").expanduser()))
client.wait_for_engines(NODES - 1)

In [ ]:
%%px --local
import dataclasses
from pprint import pformat
from pathlib import Path

import jax
from jax import numpy as jnp
from jax import random
import numpy as np
from etils import epath

jax.config.update("jax_compilation_cache_dir", str(Path("~/.jax_cache").expanduser()))
if not jax.distributed.is_initialized():
    jax.distributed.initialize()
print(jax.devices())

from deepseek_r1_jax import model as dsjax
from deepseek_r1_jax import chkpt_utils as utils

In [ ]:
%%px --local
def encode_input(tokenizer, texts, pad_id: int = 0):
    assert isinstance(texts, list)
    inputs = [
        tokenizer.apply_chat_template([{"role": "user", "content": text}]) + tokenizer.encode("<|Assistant|><think>")
        for text in texts
    ]
    max_len = max([len(x) for x in inputs])
    inputs = [(max_len - len(x)) * [pad_id] + x for x in inputs]
    return np.array(inputs)

In [ ]:
%%px --local
ckpt_path = epath.Path(f"~/bucket/deepseek-r1-jax-chkpt").expanduser()
tokenizer = dsjax.load_tokenizer()
mesh = jax.make_mesh((1, 4, jax.device_count() // 4), ("x", "y", "z"), devices=jax.devices())
cfg = dataclasses.replace(dsjax.Config(), mesh=mesh)
weights = utils.load_model(epath.Path(ckpt_path).expanduser(), cfg)

In [ ]:
%%px --local
input = encode_input(
    tokenizer,
    [
        "Tell me your name",
        "What is the weather like expressed in long prose in Old English",
        "Do you like ice cream, be extremely precise",
    ],
)

zero_cache = dsjax.KVCache.init(random.key(1), cfg, input.shape[0], cfg.max_seq_len)
curr_tokens, logits, cache = dsjax.prefill(input, weights, zero_cache, cfg)
curr_tokens, tokens_list = curr_tokens[:, cache.length - 1 : cache.length], []
tokens_list = []
for _ in range(32):
    tokens_list.append(curr_tokens)
    curr_tokens, cache = dsjax.decode_step(curr_tokens, weights, cache, cfg)
tokens = np.array(jnp.concatenate(tokens_list, axis=-1))
responses = [tokenizer.decode(row) for row in tokens]
print("Responses:\n" + pformat(responses))

: 